In [1]:
from nltk.corpus import stopwords,twitter_samples

In [2]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer

In [3]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\twitter_samples.zip.


True

In [4]:
from os import getcwd

In [5]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [7]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

In [9]:
len(positive_tweets)

5000

In [10]:
train_pos = positive_tweets[:4500]
train_neg = negative_tweets[:4500]
val_pos = positive_tweets[4500:]
val_neg = negative_tweets[4500:]

train_x = train_pos+train_neg
val_x = val_pos+val_neg

train_y = np.append(np.ones((4500)),np.zeros((4500)))
val_y = np.append(np.ones((500)),np.zeros((500)))

In [11]:
print(train_x[2438])

Great way to start the day!  :D http://t.co/N5nOp3FfnX


In [12]:
from nltk.stem import PorterStemmer

In [13]:
import re

In [23]:
def process_tweet(tweet):
    
    stemmer  = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'RT[\s]+','',tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*','',tweet)
    tweet = re.sub(r'#','',tweet)
    
    tokenizer = TweetTokenizer(preserve_case = False,strip_handles = True,reduce_len = True)
    
    tweet_tokens = tokenizer.tokenize(tweet)
    
    clean_tweets = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation):
            clean_word = stemmer.stem(word)
            clean_tweets.append(clean_word)
    
    return clean_tweets

In [24]:
def count_frequency(tweets,y):
    
    y_list = np.squeeze(y).tolist()
    
    freqs = {}
    
    for label,tweet in zip(y_list,tweets):
        for word in process_tweet(tweet):
            
            pair = (word,label)
            if pair not in freqs:
                freqs[pair]=1
            else:
                freqs[pair]+=1
                
    return freqs

In [27]:
frequencies = {}
tweets = ['I am Happy!','I am sad', 'I am worried','I love pizza','I hate burger','He is sad']
y = [1,0,0,1,0,0]
count_frequency(tweets,y)

{('happi', 1): 1,
 ('sad', 0): 2,
 ('worri', 0): 1,
 ('love', 1): 1,
 ('pizza', 1): 1,
 ('hate', 0): 1,
 ('burger', 0): 1}

In [39]:
def naive_bayes_algo(freqs,x_train,y_train):
    
    logprior = 0
    loglikelihood = {}
    vocab = set(pair[0] for pair in freqs.keys())
    V = len(vocab)
    
    N_pos = 0
    N_neg = 0
    
    for pair in freqs.keys():
        if pair[1] == 1:
            N_pos+=freqs[pair]
        else:
            N_neg+=freqs[pair]
    
    l = len(y_train)
    
    l_pos = np.sum(y_train == 1)
    
    l_neg = l - l_pos
    
    logprior = np.log(l_pos) - np.log(l_neg)
    
    for word in vocab:
        freq_in_pos_tweet = freqs.get((word,1),0)
        freq_in_neg_tweet = freqs.get((word,0),0)
        
        p_w_pos = (freq_in_pos_tweet+1)/(N_pos+V)
        p_w_neg = (freq_in_neg_tweet+1)/(N_neg+V)
        
        loglikelihood[word] = np.log(p_w_pos) - np.log(p_w_neg)
        
    return logprior,loglikelihood

In [36]:
def naive_bayes_calculate(tweet,logprior,loglikelihood):
    
    processed_tweet = process_tweet(tweet)
    probab = 0
    probab+=logprior
    
    for word in processed_tweet:
        
        if word in loglikelihood:
            
            probab+=loglikelihood[word]
    
    if probab>0:
        return 'Positive Sentiment'
    
    return 'Negative Sentiment'        
    

In [31]:
print('I love Pizza',logprior,loglikelihood)

NameError: name 'logprior' is not defined

In [37]:
freqs = count_frequency(train_x,train_y)

In [40]:
logprior,loglikelihood = naive_bayes_algo(freqs,train_x,train_y)

In [42]:
print(naive_bayes_calculate('I hate Pizza',logprior,loglikelihood))

Negative Sentiment


In [50]:
print(naive_bayes_calculate('Today is wonderful',logprior,loglikelihood))

Positive Sentiment


In [57]:
def test_algo(test_x,test_y,logprior,loglikelihood):
    accuracy = 0
    y_predicted = []
    
    for tweet in test_x:
        
        if naive_bayes_calculate(tweet,logprior,loglikelihood)[0] == 'P':
            y_predicted.append(1)
        else:
            y_predicted.append(0)

    error = []
    error = (abs(y_predicted - test_y))
    
    accuracy = np.mean((1 - error))
    
    return accuracy

In [58]:
print(test_algo(val_x,val_y,logprior,loglikelihood))

0.995
